In [1]:
import sys
import pandas as ps
sys.path.append('../batcher/utils.py')
from utils import *
import numpy as np

In [2]:
df = ps.read_csv('/home/pape/Downloads/onas/05/05062021.CSV')


In [3]:
df.columns=header_onas

In [4]:
df = filter_puissance(df)

In [5]:
df.e_active = df.e_active * -1

In [6]:
df.e_active

0          -149868
1          -149868
23     -4110418505
50     -3489661513
53         -149932
           ...    
1222       -152374
1223       -152374
1224       -152374
1225       -152374
1227       -152375
Name: e_active, Length: 356, dtype: int64

In [7]:
df['datetime'] = ps.to_datetime(df.date+" " + df.heure)

In [8]:
df.index = df.datetime
df.facteur_c = 100

In [15]:
def get_energies(df):
    df.e_active = df.e_active * -1
    energie = df.resample('1H').agg({'e_active':'sum'}).dropna() 
    energie = energie.e_active * -1
    return energie

datetime
2021-05-06 00:00:00    11860697368
2021-05-06 01:00:00      505828462
2021-05-06 02:00:00         150139
2021-05-06 03:00:00      690421039
2021-05-06 04:00:00     3528852383
2021-05-06 05:00:00    28616746271
2021-05-06 06:00:00    12637161303
2021-05-06 07:00:00    20971524704
2021-05-06 08:00:00     4044215430
2021-05-06 09:00:00      371665119
2021-05-06 10:00:00     3408647284
2021-05-06 11:00:00     3473037802
2021-05-06 12:00:00     4279249525
2021-05-06 13:00:00      825164369
2021-05-06 14:00:00      538082876
2021-05-06 15:00:00      238002717
2021-05-06 16:00:00     2890377662
2021-05-06 17:00:00              0
2021-05-06 18:00:00              0
2021-05-06 19:00:00        5624350
2021-05-06 20:00:00         456342
2021-05-06 21:00:00        1065428
2021-05-06 22:00:00        1676106
Freq: H, Name: e_active, dtype: int64

In [ ]:
#df.head(1)

In [ ]:
#df.last('H')

In [ ]:
#conso = df.energie_active_secteur.last('H')- df.energie_active_secteur[0]

In [ ]:
#conso

In [ ]:
#total = df.energie_active_secteur.aggregate(np.diff)

In [ ]:
#total

In [ ]:
eneg = df.e_active.groupby([df.index.hour]).diff().agg(np.sum)

In [ ]:
eneg

In [ ]:
def calcul_montant(df,energie,heure):
    code = df.code[0]
    tarif_1er_tranche = df.tarif_1er_tranche[0]
    tarif_2eme_tranche = df.tarif_2eme_tranche[0]
    tarif_3eme_tranche = df.tarif_3eme_tranche[0]
    tarif_heure_pleine = df.tarif_heure_pleine[0]
    tarif_heure_creuse = df.tarif_heure_creuse[0]
    max_1er_tranche = df.max_1er_tranche[0]
    max_2eme_tranche = df.max_2eme_tranche[0]
    facteur_c = df.facteur_c[0]
    id_compteur = df.id_compteur[0]
    if code == 'GP':
        if (heure >=19) & (heure<=23):
            montant = energie /facteur_c * tarif_heure_pleine
            return montant
        else:
            montant  = energie /facteur_c * tarif_heure_creuse
            return montant
    else:
        # get redis energie we will call it conso_redis
        conso_redis = redis.get(id_compteur)
        conso_total_mois_en_cours = conso_redis + energie
        if conso_total_mois_en_cours < max_1er_tranche:
            montant = energie /facteur_c * tarif_1er_tranche
            return montant
        elif (conso_total_mois_en_cours >=max_1er_tranche) & (conso_total_mois_en_cours<=max_2eme_tranche):
            montant = energie /facteur_c * tarif_2eme_tranche
            return montant
        else:
            montant = energie/facteur_c*tarif_3eme_tranche
            return montant
        
            
        
    

In [ ]:
def make_stats(df):
    df.intensite_moyen = df.intensite.aggregate(np.mean)
    df.intensite_max = df.intensite.aggregate(np.max)
    df.intensite_min = df.intensite.aggregate(np.min)
    df.tension_moyen = df.ul1.aggregate(np.min)
    df.tension_min = df.ul1.aggregate(np.min)
    df.tension_max = df.ul1.aggregate(np.max)
    df.puissance_maxi= df.p_1.aggregate(np.max)
    df.puissance_min = df.p_1.aggregate(np.min)
    df.puissance_moyen = df.p_1.aggregate(np.mean)
    # use the resample method 
    
    